# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [2]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

!cp /usr/lib/chromium-browser/chromedriver /usr/bin/

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [49.4 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:15 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [603 kB]
Get:11 http://archive.ubuntu

In [3]:
import pandas as pd
import time
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('-no-sandbox')
options.add_argument('-headless')
options.add_argument('-disable-dev-shm-usage')
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.common.by import By
driver = webdriver.Chrome('chromedriver',options=options)
link = 'https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv'
main_arr = []
sub_arr = []
driver.get(link)
for num in range(3):
  driver.find_element_by_class_name("ipl-load-more__button").click()
  time.sleep(6)
  list1 = driver.find_elements(By.CLASS_NAME, "title")
  list2 = driver.find_elements(By.CLASS_NAME, "text")
for x, y in zip(list1, list2):
      main_arr.append((x.text).replace('\n',''))
      sub_arr.append(y.text)
df = pd.DataFrame(list(zip(main_arr, sub_arr)), columns =['Title of product', 'Detailed Review'])
print("Length of data frame is {0}".format(len(df)))
df




Length of data frame is 100


,Title of product,Detailed Review
0,Outstanding movie with a haunting performance ...,"Every once in a while a movie comes, that trul..."
1,Only certain people can relate,This is a movie that only those who have felt ...
2,Perfect in every aspect.,"Truly a masterpiece, The Best Hollywood film o..."
3,MASTERPIECE 😍,Joaquin Phoenix gives a tour de force performa...
4,The Hype is real,Most of the time movies are anticipated like t...
...,...,...
95,What a joke,"Heath great in dark Knight, even liked Jared L..."
96,Glad I didn't waste my money on this movie,I was very excited about this movie when it ca...
97,Derivative and Uninspired,There is doing an homage and then there is bor...
98,Better if didn't saw,Disturbing. Just got out of the movie. My humo...


In [4]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package brown to /root/nltk_data...
       |   Unzipping corpora/brown.zip.
       | Downloading package brown_tei to /root/nltk_data...
       |   Unzipping corpora/brown_tei.zip.
       | Downloading package cess_cat to /root/nltk_data...
       |   Unzipping corpora/cess_cat.zip.
       | Downloading package cess_esp to /root/nltk_data...
 

True

**Dataset** **Modification** **as** **assign2** 

In [5]:
import re
from nltk.corpus import stopwords
y = stopwords.words('english')
df['Stopwords Removal'] = df['Detailed Review'].apply(lambda x: " ".join(x for x in x.split() if x not in y))
df['Lower Case'] = df['Stopwords Removal'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['Punctuation Removal'] = df['Lower Case'].str.replace('[^\w\s]','')
df['Special Characters Removal'] = df['Punctuation Removal'].apply(lambda x: ''.join(re.sub(r"[^a-zA-Z0-9]+", ' ', charctr) for charctr in x ))
from textblob import TextBlob
df['Spelling Correction'] = df['Special Characters Removal'].apply(lambda x: str(TextBlob(x).correct()))




In [6]:
import nltk
nltk.download('punkt')
df['Tokenization'] = df['Spelling Correction'].apply(lambda x: TextBlob(x).words)
from nltk.stem import PorterStemmer
ps = PorterStemmer()
df['Stemming'] = df['Tokenization'].apply(lambda x: " ".join([ps.stem(word) for word in x]))
from textblob import Word
import nltk
nltk.download('wordnet')
df['Lemmatization'] = df['Stemming'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
df.to_csv('/content/sample_data/assign5_review.csv',index=False)

**N-grams:**

In [8]:
import itertools
import collections
from nltk import ngrams
from nltk.tokenize import word_tokenize

array = []
for text in df['Lemmatization']:
  array.append(word_tokenize(text))
data_process = [x for x in array if x != []]
y = list(itertools.chain.from_iterable(data_process))
tgram = nltk.trigrams(y)
fq = nltk.FreqDist(tgram)
fq

FreqDist({('everi', 'movi', 'come'): 1,
          ('movi', 'come', 'truli'): 1,
          ('come', 'truli', 'make'): 1,
          ('truli', 'make', 'impact'): 1,
          ('make', 'impact', 'joaquin'): 1,
          ('impact', 'joaquin', 'perform'): 1,
          ('joaquin', 'perform', 'scenographi'): 1,
          ('perform', 'scenographi', 'brillianc'): 1,
          ('scenographi', 'brillianc', 'grotesqu'): 1,
          ('brillianc', 'grotesqu', 'hunt'): 1,
          ('grotesqu', 'hunt', 'cring'): 1,
          ('hunt', 'cring', 'hard'): 1,
          ('cring', 'hard', 'watch'): 1,
          ('hard', 'watch', 'time'): 1,
          ('watch', 'time', 'mesmer'): 1,
          ('time', 'mesmer', 'blink'): 1,
          ('mesmer', 'blink', 'eye'): 1,
          ('blink', 'eye', 'watch'): 1,
          ('eye', 'watch', 'it'): 1,
          ('watch', 'it', 'tragic'): 1,
          ('it', 'tragic', 'serious'): 1,
          ('tragic', 'serious', 'funni'): 1,
          ('serious', 'funni', 'moment'): 1,

**Probability of bigrams in dataset with the given formula**

In [10]:
from collections import Counter
bigrams = nltk.bigrams(y)
fq = nltk.FreqDist(bigrams)
dic = dict(fq)
for data in dic:
  print( str(data) + ':' + str(dic[data] / y.count(data[0])))

Streaming output truncated to the last 5000 lines.
('the', 'movi'):0.07446808510638298
('movi', 'affect'):0.004975124378109453
('affect', 'way'):1.0
('way', 'make'):0.03571428571428571
('make', 'physic'):0.034482758620689655
('physic', 'pain'):0.25
('pain', 'experi'):0.2
('experi', 'good'):0.25
('good', 'way'):0.022222222222222223
('way', 'need'):0.03571428571428571
('need', 'i'):0.05555555555555555
('i', 'say'):0.03488372093023256
('say', 'more'):0.034482758620689655
('more', 'everyth'):0.1
('everyth', 'movi'):0.2
('movi', 'wast'):0.004975124378109453
('wast', 'everi'):0.2
('everi', 'singl'):0.2857142857142857
('singl', 'way'):0.25
('way', 'joke'):0.03571428571428571
('joke', 'awesom'):0.008
('awesom', 'comic'):0.5
('book', 'movi'):0.25
('movi', 'awesom'):0.004975124378109453
('awesom', 'movi'):0.5
('movi', 'period'):0.004975124378109453
('period', 'it'):1.0
('it', 'offer'):0.010869565217391304
('offer', 'easi'):0.3333333333333333
('easi', 'answer'):0.3333333333333333
('answer', 'sett

**Revewing Noun phrases and relative probabilities**

In [22]:
from textblob import TextBlob
set = ''
value = []
ite = 1
for x in df['Lemmatization']:
  set = set + x
  result = 'Review-' + str(ite)
  value.append(result)
  ite += 1
noun_phrases = []
fq = []
for x in df['Lemmatization']:
  blob = TextBlob(x)
  for nouns in blob.noun_phrases:
    noun_phrases.append(nouns)
for w in noun_phrases:
  noun_phrases_f = []
  for x in df['Lemmatization']:
    noun_phrases_f.append(x.count(w) /set.count(w))
  fq.append(noun_phrases_f)
noun_phrases_df = pd.DataFrame(fq).T
noun_phrases_df.columns = list(noun_phrases)
noun_phrases_df.value = value
noun_phrases_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


,everi movi,impact joaquin,scenographi brillianc grotesqu hunt,time mesmer blink eye,serious funni moment emot rollercoast sometim multipl emot,typic actionriddl,superhero movi,proper psycholog thrillerdrama singl,thi movi,alon isol truli relat,sorri charact,lot peopl,encourag violenc,truli movi encourag everi,u becom,everyon respect,truli masterpiec,hollywood film,film decad,comic book willingli realist,direct cinematographi music act,peopl surpris,violent necess messag,societi reflect underappreciatedunrecognizedbulli peopl,way show class differ,corrupt rich talent rule,multipl joke,bitter way film show,peopl comfort peopl consid wake,perfect film,joaquin phoenix,fearless stun emot depth physic,referenc heath ledger oscarwin,dark knight,wide consid definit liveact portray joke,stun phoenix,imposs portray,potenti exce,dark knight clown princ crime,end fall,...,everyon el,great dark knight,suicid squad,joke definit,wast money cinema phoenix depress,cup tea person,charact tongu cheek type harvey,i excit movi,theater rave review i,excit i,glad i didn thi movi,flat bore plot liter noth,insecur man talk therapist laugh,peopl arthur laugh terribl annoy,whole movi arthur depth person thi movi bore i,fall asleep anyon thing movi,meaning masterpiec clearli,i list mani,deep thi,movi tri show villain,comic distraught weak man,realiti joke,comic anyth,crazi compet master,oscar nomin idiot movi anyth oscar worthi,homag borrow wholesal materi thi film littl,taxi driver element,notabl end borrow network,case taxi driver,whole line dialogu charact scene,place movi,execut polish act calibr stori origin,taxi driver network film,piti guy suffer lack charm,killer heath ledger,legendari charact legendari,actor i abl,heath ledger,movi total,thi movi deserv oscar
0,1.0,1.0,1.0,1.0,1.0,1.0,0.2,1.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.05,1.0,1.0,0.5,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.037037,1.0,1.0,0.222222,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.071429,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,

# **Question 2: Undersand TF-IDF and Document representation**

(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [36]:
import math
def tf_idf_value(sent,text):
  x = len(sent.split(" "))
  tval = sent.count(text)/x
  idfval = 0
  if(tval!=0):
    idfval = math.log(x)/sent.count(text)
  else:
    return 0;
  return tval*idfval
s = df["Spelling Correction"].values.tolist()
tk = list([j for i in s for j in i.split(" ")])
res = pd.DataFrame(tk,columns=["token"])
count = 0
for i in s:
  res[str(count)] = res["token"].apply(lambda x : tf_idf_value(i,x))
  count+=1
res

,token,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,every,0.07824,0.086004,0.000000,0.072861,0.000000,0.04957,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.072861,0.036551,0.0,0.000000,0.0,0.052748,0.000000,0.052748,0.000000,0.019607,0.0,0.000000,0.000000,0.0,0.0,0.061364,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.0,0.090575,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.060693,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.060693,0.000000,0.000000,0.000000,0.000000,0.000000
1,movie,0.07824,0.086004,0.000000,0.000000,0.072861,0.04957,0.092222,0.032042,0.062757,0.0,0.0,0.054775,0.173287,0.230259,0.072861,0.036551,0.0,0.000000,0.0,0.052748,0.070008,0.052748,0.000000,0.019607,0.0,0.034513,0.061364,0.0,0.0,0.000000,0.00000,0.000000,0.00000,0.00000,0.069111,0.101581,0.0,0.090575,0.0,...,0.000000,0.064983,0.083231,0.031405,0.0,0.000000,0.032538,0.067391,0.013704,0.060693,0.0,0.070931,0.017796,0.180537,0.188504,0.197304,0.160576,0.092222,0.000000,0.026385,0.103716,0.000000,0.056413,0.043671,0.044658,0.277987,0.119007,0.132419,0.000000,0.057567,0.000000,0.000000,0.108304,0.000000,0.000000,0.000000,0.043353,0.081918,0.132419,0.116114
2,comes,0.07824,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,truly,0.07824,0.086004,0.051332,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,makes,0.07824,0.000000,0.051332,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.054775,0.000000,0.230259,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.060693,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6479,origin,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.

**cosine similarity Ranking**

In [63]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
query = 'An emotional movie with a top notch performance and the scences potrayed by Joaquin Phoenix was a masterpeice. well deployed script with engaging screenplay'
x = word_tokenize(query)
sw = stopwords.words('english') 
xs = {w for w in x if not w in sw} 
def vec(xs, ys):
  rvec = xs.union(ys)
  lis1 =[];lis2 =[]
  for w in rvec: 
      if w in xs: lis1.append(1)
      else: lis1.append(0) 
      if w in ys: lis2.append(1) 
      else: lis2.append(0)
  return rvec, lis1, lis2
def cc(rvec, lis1, lis2):
  con = 0
  for i in range(len(rvec)): 
        con+= lis1[i]*lis2[i] 
  cos = con / float((sum(lis1)*sum(lis2))**0.5) 
  return cos
sl = []
for sent in df['Spelling Correction']:
  yl = word_tokenize(sent)
  yset = {w for w in yl if not w in sw}
  rvector, l1, l2 = compute_vector(X_set, yset)
  try:
    sim = cc(rvector, l1, l2)
  except ZeroDivisionError:
    sim = 'None'
  sl.append(sim)
cdf = pd.DataFrame(list(zip(df['Detailed Review'],sl)), columns=['Review ',' Ranking Cosine Similarity'])
cdf


,Review,Ranking Cosine Similarity
0,"Every once in a while a movie comes, that trul...",0.108465
1,This is a movie that only those who have felt ...,0.04222
2,"Truly a masterpiece, The Best Hollywood film o...",0
3,Joaquin Phoenix gives a tour de force performa...,0.0748481
4,Most of the time movies are anticipated like t...,0.0378777
...,...,...
95,"Heath great in dark Knight, even liked Jared L...",0.0342997
96,I was very excited about this movie when it ca...,0.0283866
97,There is doing an homage and then there is bor...,0.0404226
98,Disturbing. Just got out of the movie. My humo...,0.111283


# **Question 3: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [ ]:
https://github.com/santhanreddy864/In-class-exercise-1/blob/main/assignment5_review.csv